In [8]:
# Imports
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
# Create dataframe
df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')
print(df)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  Outcome  
0                       0.627   50        1  
1                  

In [3]:
# Separate the features and labels
X = df.drop('Outcome', axis=1)
y = df['Outcome']

In [9]:
# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=32)

# Perform train-validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=32)

In [10]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
# Build the MLP model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=4, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(units=2, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define the learning rate reduction callback
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',    # Monitor validation loss for learning rate reduction
    factor=0.5,            # Reduce learning rate by a factor
    patience=10,            # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6            # Minimum learning rate
)

# Define the model checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Define the early stopping callback to stop training if validation loss does not improve
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)

# Train the model with the learning rate reduction callback
model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=20, callbacks=[lr_callback,checkpoint_callback,early_stopping_callback])

# Evaluate the model
best_model = tf.keras.models.load_model('best_model.h5')
loss, accuracy = best_model.evaluate(X_test, y_test)
print('Loss = {} \nAccuracy = {}'.format(loss, accuracy*100))

Epoch 1/20
18/18 [==============================] - 1s 15ms/step - loss: 0.6867 - accuracy: 0.5812 - val_loss: 12.7729 - val_accuracy: 0.6429 - lr: 0.0010
Epoch 2/20
18/18 [==============================] - 0s 5ms/step - loss: 0.6660 - accuracy: 0.6083 - val_loss: 10.7604 - val_accuracy: 0.6429 - lr: 0.0010
Epoch 3/20
18/18 [==============================] - 0s 5ms/step - loss: 0.6502 - accuracy: 0.6227 - val_loss: 8.9848 - val_accuracy: 0.6429 - lr: 0.0010
Epoch 4/20
18/18 [==============================] - 0s 4ms/step - loss: 0.6349 - accuracy: 0.6408 - val_loss: 7.4911 - val_accuracy: 0.6531 - lr: 0.0010
Epoch 5/20
18/18 [==============================] - 0s 5ms/step - loss: 0.6225 - accuracy: 0.6516 - val_loss: 6.1689 - val_accuracy: 0.6633 - lr: 0.0010
Epoch 6/20
18/18 [==============================] - 0s 5ms/step - loss: 0.6108 - accuracy: 0.6552 - val_loss: 5.1034 - val_accuracy: 0.6837 - lr: 0.0010
Epoch 7/20
18/18 [==============================] - 0s 5ms/step - loss: 0.6006 